In [1]:
import sys, os, glob
from tqdm import tqdm
import cv2
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
sys.path.append('/users/rhydian/self-supervised-project')

from keypoint_alignment_dataset import KeypointAlignmentDataset
from models.SSECEncoders import VGGEncoder, HighResVGGEncoder
from train_SSEC import load_models
i
from registration import get_matching_keypoints_correspondance, get_SIFT_correspondance, get_dense_correspondance, transform_points, find_best_rigid, resample_point
from plotting import show_matches, show_warped_perspective, show_transformed_keypoints
import torch.nn.functional as F

In [2]:
class config():
    TRAINING_AUGMENTATION=False
    DATASET_ROOT = '/scratch/shared/beegfs/rhydian/UKBiobank'
    #MODEL_LOAD_PATH = '/users/rhydian/self-supervised-project/model_weights/SSECEncodersHighRes2'#     
    MODEL_LOAD_PATH = '/users/rhydian/self-supervised-project/model_weights/SSECEncodersBothBoth'
    USE_EQUAL_RES = True
    # ENCODER_TYPE = HighResVGGEncoder
    ENCODER_TYPE = VGGEncoder
    EMBEDDING_SIZE = 128
    USE_CUDA = False 
    UPSAMPLE_FACTOR = 2 # amount feature maps scaled up for subpixel accuracy
    USE_CYCLIC = True # use cyclic point correlation
    USE_RANSAC = True
    RANSAC_TOLERANCE = 15
    GRID_RES = 1
    ALLOW_SCALING = False
    NUM_DISCRIM_POINTS = int(200*GRID_RES**2) # number of correlating pairs to find
    METHOD = 'MATCH_KEYPOINT' # from DENSE, SIFT, MATCH_KEYPOINT
    assert METHOD in ['MATCH_KEYPOINT','DENSE','SIFT']


def set_up():
    global c
    ds  = KeypointAlignmentDataset()
    dxa_model, mri_model, val_stats, epochs = load_models(c.ENCODER_TYPE, c.EMBEDDING_SIZE, 
                                                          c.MODEL_LOAD_PATH, c.USE_CUDA, SINGLE_MRI=0, SINGLE_DXA=0 )
    return ds, dxa_model, mri_model, val_stats, epochs

c = config()
ds, dxa_model, mri_model, val_stats, epochs = set_up()

Trying to load from /users/rhydian/self-supervised-project/model_weights/SSECEncodersBothBoth
==> Resuming model trained for 106 epochs...


/users/rhydian/anaconda3/envs/torch_env/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729006826/work/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
